# Programmazione Concorrente e Parallela

Module: M02056.2  
Course: C02045.2    
Teachers: Tiziano Leidi, Loris Grossi, Diego Frei

## Threads

### Possibilità di utilizzo
#### Estensione della classe Thread

In [1]:
class MyThread extends Thread {
    public void run() {
        System.out.println("Esecuzione Task #1");
    }
}
Thread thread1 = new MyThread();
thread1.start();


Esecuzione Task #1

#### Utilizzo di un Anonymous Class

In [2]:
Thread thread2 = new Thread(new Runnable() {
    public void run() {
        System.out.println("Esecuzione Task #2");
    }
});
thread2.start();

Esecuzione Task #2


#### Utilizzo di una Lambda Expression

In [3]:
Thread thread3 = new Thread(() -> System.out.println("Esecuzione Task #3"));
thread3.start();

Esecuzione Task #3

### Funzionalità dei threads

#### Sleep
Metodi statici che mettono a dormire il thread corrente per l’intervallo di tempo specificato (o più dell’intervallo, a seconda di quando il thread verrà ri-schedulato).

In [9]:
Thread thread = new Thread(() -> System.out.println("Esecuzione Task dopo 200ms"));
thread.sleep(200); // Sleep for 1s
thread.start();

Esecuzione Task dopo 200ms


#### Join

Mette ad aspettare il thread corrente finché il thread specificato non ha terminato la sua esecuzione.

In [19]:
Thread thread1 = new Thread(() -> {
    System.out.println("Task 1");
    try{
        Thread.sleep(100);
    }
    catch(InterruptedException e){
        
    }
});
Thread thread2 = new Thread(() -> {
    System.out.println("Task 2");
    try{
        Thread.sleep(200);
    }
    catch(InterruptedException e){
        
    }
});

thread1.start();
thread2.start();

thread1.join();
thread2.join();
System.out.println("Tutti i task hanno terminato la loro esecuzione");

Task 1
Task 2
Tutti i task hanno terminato la loro esecuzione


### Esempio

In [23]:
final Thread[] threads = new Thread[10];
for (int i = 0; i < threads.length; i++) {
    threads[i] = new Thread(new Runnable() {
    public void run() {
        System.out.println("Thread started");
        try {
            Thread.sleep((long) Math.random() * 1000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.println("Thread completed");
    }
    });
    threads[i].start();
}
for (int i = 0; i < threads.length; i++) {
    try {
        threads[i].join();
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
}
System.out.println("Thread all completed");

Thread started
Thread started
Thread completed
Thread started
Thread completed
Thread completed
Thread started
Thread completed
Thread started
Thread started
Thread completed
Thread started
Thread started
Thread started
Thread completed
Thread completed
Thread started
Thread completed
Thread completed
Thread completed
Thread all completed


## Thread Safety

Thread-safe: Il programma ha un comportamento corretto quando è eseguito su più thread  
I programmi single-thread non hanno bisogno di essere thread-safe.

### Memoria condivisa
